In [17]:
import pandas as pd
import requests
import time
from typing import Dict, List, Optional

class PertaminaAPIScraper:
    def __init__(self):
        self.base_url = "https://subsiditepatlpg.mypertamina.id/infolpg3kg/api"
        self.headers = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
            "Accept": "application/json",
            "Content-Type": "application/json"
        }
    
    def get_merchants(self, village_id: str, page: int = 1, size: int = 100) -> Optional[Dict]:
        try:
            url = f"{self.base_url}/general/users/v1/merchants"
            params = {
                "page": page,
                "size": size,
                "villageId": village_id
            }
            
            response = requests.get(url, params=params, headers=self.headers)
            time.sleep(1)  # Delay untuk menghindari rate limiting
            
            if response.status_code == 200:
                return response.json()
            else:
                print(f"Error {response.status_code}: {response.text}")
                return None
                
        except Exception as e:
            print(f"Error getting merchants: {str(e)}")
            return None

def analyze_and_update_coordinates():
    """
    Menganalisis data koordinat NULL dan mengupdate menggunakan API
    """
    try:
        # Baca CSV dengan separator semicolon
        df = pd.read_csv('merchants_tangsel.csv', 
                        sep=';',
                        encoding='utf-8',
                        on_bad_lines='warn')
        
        print("\n=== ANALISIS DATA AWAL ===")
        print(f"Total data: {len(df)}")
        
        # Bersihkan data koordinat
        df['Koordinat'] = df['Koordinat'].astype(str).str.replace('"', '').str.strip()
        
        # Pisahkan koordinat menjadi latitude dan longitude
        df[['Latitude', 'Longitude']] = df['Koordinat'].str.split(',', expand=True)
        
        # Bersihkan nilai latitude dan longitude
        df['Latitude'] = df['Latitude'].str.replace('[^\d.-]', '', regex=True)
        df['Longitude'] = df['Longitude'].str.replace('[^\d.-]', '', regex=True)
        
        # Konversi ke float
        df['Latitude'] = pd.to_numeric(df['Latitude'], errors='coerce')
        df['Longitude'] = pd.to_numeric(df['Longitude'], errors='coerce')
        
        # Identifikasi data NULL
        null_coords = df[df['Latitude'].isna() | df['Longitude'].isna()].copy()
        print(f"\nData dengan koordinat NULL: {len(null_coords)}")
        
        if len(null_coords) > 0:
            print("\nMemulai proses update koordinat dari API...")
            
            # Inisialisasi scraper
            scraper = PertaminaAPIScraper()
            updated_count = 0
            
            # Dapatkan data dari API untuk setiap kelurahan
            unique_villages = null_coords['Kelurahan'].unique()
            
            for village in unique_villages:
                print(f"\nMengambil data untuk kelurahan: {village}")
                
                # Gunakan village ID sesuai dengan data Anda
                village_id = "608605d4-7864-11ed-8cf6-4201c0a80002"  # Sesuaikan dengan ID yang benar
                
                # Ambil data merchants
                page = 1
                merchant_data = {}
                
                while True:
                    result = scraper.get_merchants(village_id, page)
                    if not result or not result.get('content'):
                        break
                    
                    # Proses setiap merchant
                    for merchant in result['content']:
                        merchant_name = merchant.get('name', '').upper()
                        if merchant.get('latitude') and merchant.get('longitude'):
                            merchant_data[merchant_name] = {
                                'latitude': merchant['latitude'],
                                'longitude': merchant['longitude']
                            }
                    
                    if page >= result.get('totalPages', 1):
                        break
                    page += 1
                
                # Update koordinat untuk merchants di kelurahan ini
                village_nulls = null_coords[null_coords['Kelurahan'] == village]
                for idx in village_nulls.index:
                    merchant_name = df.loc[idx, 'Nama Pemilik']
                    if merchant_name in merchant_data:
                        df.loc[idx, 'Latitude'] = merchant_data[merchant_name]['latitude']
                        df.loc[idx, 'Longitude'] = merchant_data[merchant_name]['longitude']
                        updated_count += 1
            
            print(f"\nBerhasil mengupdate {updated_count} dari {len(null_coords)} data koordinat NULL")
            
            # Update kolom Koordinat dengan format yang benar
            df['Koordinat'] = df.apply(
                lambda x: f"{x['Latitude']}, {x['Longitude']}" if pd.notnull(x['Latitude']) and pd.notnull(x['Longitude']) else x['Koordinat'],
                axis=1
            )
            
            # Simpan hasil
            df.to_csv('merchants_tangsel_updated.csv', sep=';', index=False)
            print("\nData telah disimpan ke merchants_tangsel_updated.csv")
            
            # Simpan data yang masih NULL untuk pengecekan manual
            still_null = df[df['Latitude'].isna() | df['Longitude'].isna()]
            if len(still_null) > 0:
                still_null.to_csv('koordinat_masih_null.csv', sep=';', index=False)
                print(f"\nMasih terdapat {len(still_null)} data dengan koordinat NULL")
                print("Data tersebut telah disimpan ke koordinat_masih_null.csv")
        
    except Exception as e:
        print(f"Error dalam pemrosesan data: {str(e)}")

if __name__ == "__main__":
    analyze_and_update_coordinates()

<>:58: SyntaxWarning: invalid escape sequence '\d'
<>:59: SyntaxWarning: invalid escape sequence '\d'
<>:58: SyntaxWarning: invalid escape sequence '\d'
<>:59: SyntaxWarning: invalid escape sequence '\d'
C:\Users\user\AppData\Local\Temp\ipykernel_10552\632736589.py:58: SyntaxWarning: invalid escape sequence '\d'
  df['Latitude'] = df['Latitude'].str.replace('[^\d.-]', '', regex=True)
C:\Users\user\AppData\Local\Temp\ipykernel_10552\632736589.py:59: SyntaxWarning: invalid escape sequence '\d'
  df['Longitude'] = df['Longitude'].str.replace('[^\d.-]', '', regex=True)



=== ANALISIS DATA AWAL ===
Total data: 1088

Data dengan koordinat NULL: 293

Memulai proses update koordinat dari API...

Mengambil data untuk kelurahan: CIPAYUNG

Mengambil data untuk kelurahan: CIPUTAT

Mengambil data untuk kelurahan: JOMBANG

Mengambil data untuk kelurahan: SAWAH

Mengambil data untuk kelurahan: SAWAH BARU

Mengambil data untuk kelurahan: SERUA

Mengambil data untuk kelurahan: SERUA INDAH

Mengambil data untuk kelurahan: CEMPAKA PUTIH

Mengambil data untuk kelurahan: CIREUNDEU

Mengambil data untuk kelurahan: PISANGAN

Mengambil data untuk kelurahan: PONDOK RANJI

Mengambil data untuk kelurahan: REMPOA

Mengambil data untuk kelurahan: RENGAS

Mengambil data untuk kelurahan: BAMBUAPUS

Mengambil data untuk kelurahan: BENDA BARU

Mengambil data untuk kelurahan: KEDAUNG

Mengambil data untuk kelurahan: PAMULANG BARAT

Mengambil data untuk kelurahan: PAMULANG TIMUR

Mengambil data untuk kelurahan: PONDOK BENDA

Mengambil data untuk kelurahan: PONDOK CABE ILIR

Mengamb

In [19]:
import pandas as pd
import requests
import time
from typing import Dict, Optional, Tuple
from urllib.parse import quote

class GeocodingService:
    def __init__(self):
        self.base_url = "https://nominatim.openstreetmap.org/search"
        self.headers = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
        }
    
    def get_coordinates(self, address: str, city: str = "Tangerang Selatan") -> Optional[Tuple[float, float]]:
        """
        Mendapatkan koordinat dari alamat menggunakan OpenStreetMap Nominatim
        """
        try:
            # Format alamat lengkap
            full_address = f"{address}, {city}, Banten, Indonesia"
            
            # Encode alamat untuk URL
            encoded_address = quote(full_address)
            
            # Parameters untuk request
            params = {
                "q": encoded_address,
                "format": "json",
                "limit": 1,
                "countrycodes": "id"
            }
            
            # Kirim request
            response = requests.get(
                self.base_url,
                params=params,
                headers=self.headers
            )
            
            # Delay untuk menghormati rate limit
            time.sleep(1)
            
            if response.status_code == 200:
                results = response.json()
                if results:
                    return (
                        float(results[0]["lat"]),
                        float(results[0]["lon"])
                    )
            return None
            
        except Exception as e:
            print(f"Error getting coordinates for {address}: {str(e)}")
            return None

def analyze_and_update_coordinates():
    """
    Menganalisis data koordinat NULL dan mengupdate menggunakan OpenStreetMap
    """
    try:
        # Baca CSV dengan separator semicolon
        df = pd.read_csv('merchants_tangsel.csv', 
                        sep=';',
                        encoding='utf-8',
                        on_bad_lines='warn')
        
        print("\n=== ANALISIS DATA AWAL ===")
        print(f"Total data: {len(df)}")
        
        # Bersihkan data koordinat
        df['Koordinat'] = df['Koordinat'].astype(str).str.replace('"', '').str.strip()
        
        # Pisahkan koordinat menjadi latitude dan longitude
        df[['Latitude', 'Longitude']] = df['Koordinat'].str.split(',', expand=True)
        
        # Bersihkan nilai latitude dan longitude
        df['Latitude'] = df['Latitude'].str.replace('[^\d.-]', '', regex=True)
        df['Longitude'] = df['Longitude'].str.replace('[^\d.-]', '', regex=True)
        
        # Konversi ke float
        df['Latitude'] = pd.to_numeric(df['Latitude'], errors='coerce')
        df['Longitude'] = pd.to_numeric(df['Longitude'], errors='coerce')
        
        # Identifikasi data NULL
        null_coords = df[df['Latitude'].isna() | df['Longitude'].isna()].copy()
        print(f"\nData dengan koordinat NULL: {len(null_coords)}")
        
        if len(null_coords) > 0:
            print("\nMemulai proses update koordinat dari OpenStreetMap...")
            
            # Inisialisasi geocoding service
            geocoder = GeocodingService()
            updated_count = 0
            
            # Proses setiap baris dengan koordinat NULL
            for idx in null_coords.index:
                alamat = df.loc[idx, 'Alamat']
                kecamatan = df.loc[idx, 'Kecamatan']
                
                print(f"\nMencari koordinat untuk: {alamat}, {kecamatan}")
                
                # Dapatkan koordinat
                result = geocoder.get_coordinates(
                    address=f"{alamat}, {kecamatan}"
                )
                
                if result:
                    df.loc[idx, 'Latitude'] = result[0]
                    df.loc[idx, 'Longitude'] = result[1]
                    updated_count += 1
                    print(f"Berhasil: {result[0]}, {result[1]}")
                else:
                    print("Tidak ditemukan")
            
            print(f"\nBerhasil mengupdate {updated_count} dari {len(null_coords)} data koordinat NULL")
            
            # Update kolom Koordinat dengan format yang benar
            df['Koordinat'] = df.apply(
                lambda x: f"{x['Latitude']}, {x['Longitude']}" if pd.notnull(x['Latitude']) and pd.notnull(x['Longitude']) else x['Koordinat'],
                axis=1
            )
            
            # Simpan hasil
            df.to_csv('merchants_tangsel_updated.csv', sep=';', index=False)
            print("\nData telah disimpan ke merchants_tangsel_updated.csv")
            
            # Simpan data yang masih NULL untuk pengecekan manual
            still_null = df[df['Latitude'].isna() | df['Longitude'].isna()]
            if len(still_null) > 0:
                still_null.to_csv('koordinat_masih_null.csv', sep=';', index=False)
                print(f"\nMasih terdapat {len(still_null)} data dengan koordinat NULL")
                print("Data tersebut telah disimpan ke koordinat_masih_null.csv")
            
            # Simpan log pencarian
            with open('geocoding_log.txt', 'w', encoding='utf-8') as f:
                f.write(f"Total data: {len(df)}\n")
                f.write(f"Data NULL awal: {len(null_coords)}\n")
                f.write(f"Berhasil diupdate: {updated_count}\n")
                f.write(f"Gagal diupdate: {len(null_coords) - updated_count}\n")
        
    except Exception as e:
        print(f"Error dalam pemrosesan data: {str(e)}")

if __name__ == "__main__":
    analyze_and_update_coordinates()

<>:77: SyntaxWarning: invalid escape sequence '\d'
<>:78: SyntaxWarning: invalid escape sequence '\d'
<>:77: SyntaxWarning: invalid escape sequence '\d'
<>:78: SyntaxWarning: invalid escape sequence '\d'
C:\Users\user\AppData\Local\Temp\ipykernel_10552\2109954341.py:77: SyntaxWarning: invalid escape sequence '\d'
  df['Latitude'] = df['Latitude'].str.replace('[^\d.-]', '', regex=True)
C:\Users\user\AppData\Local\Temp\ipykernel_10552\2109954341.py:78: SyntaxWarning: invalid escape sequence '\d'
  df['Longitude'] = df['Longitude'].str.replace('[^\d.-]', '', regex=True)



=== ANALISIS DATA AWAL ===
Total data: 1088

Data dengan koordinat NULL: 293

Memulai proses update koordinat dari OpenStreetMap...

Mencari koordinat untuk: JL. DUTA LESTARI, CIPUTAT
Tidak ditemukan

Mencari koordinat untuk: JL. CIMANDIRI VI, CIPUTAT
Tidak ditemukan

Mencari koordinat untuk: JL CIMANDIRI RAYA RT 001 / RW 008, CIPUTAT
Tidak ditemukan

Mencari koordinat untuk: Jl.Cendrawasih no.69, CIPUTAT
Tidak ditemukan

Mencari koordinat untuk: Jl Citarum lll, CIPUTAT
Tidak ditemukan

Mencari koordinat untuk: Jl kesehatan 2 no 53, CIPUTAT
Tidak ditemukan

Mencari koordinat untuk: Jl Cimandiri, CIPUTAT
Tidak ditemukan

Mencari koordinat untuk: Jl Makmur Rt.04 RW.05 Kel. Cipayung, CIPUTAT
Tidak ditemukan

Mencari koordinat untuk: Jl Cimandiri Raya RT.04 RW.05 Kel. Cipayung, CIPUTAT
Tidak ditemukan

Mencari koordinat untuk: Jl. Ir. H. Juanda No. 06, CIPUTAT
Tidak ditemukan

Mencari koordinat untuk: JL. CIMANDIRI RT 05 RW 05, CIPUTAT
Tidak ditemukan

Mencari koordinat untuk: JL KEBON DU

C:\Users\user\AppData\Local\Temp\ipykernel_10552\2109954341.py:77: SyntaxWarning: invalid escape sequence '\d'
  df['Latitude'] = df['Latitude'].str.replace('[^\d.-]', '', regex=True)
C:\Users\user\AppData\Local\Temp\ipykernel_10552\2109954341.py:78: SyntaxWarning: invalid escape sequence '\d'
  df['Longitude'] = df['Longitude'].str.replace('[^\d.-]', '', regex=True)


KeyboardInterrupt: 

In [26]:
import pandas as pd
import geocoder
import time
from typing import Optional, Tuple

class MultiGeocodingService:
    def __init__(self):
        self.providers = ['osm', 'bing', 'arcgis']  # Bisa ditambahkan 'google' jika punya API key
        self.delay = 1  # Delay antar request dalam detik
    
    def clean_address(self, address: str, kecamatan: str) -> str:
        """
        Membersihkan dan memformat alamat untuk meningkatkan akurasi geocoding
        """
        # Standardisasi format jalan
        address = address.replace('JL.', 'Jalan')
        address = address.replace('JL ', 'Jalan ')
        address = address.replace('Jl.', 'Jalan')
        address = address.replace('Jl ', 'Jalan ')
        
        # Hapus detail RT/RW karena bisa mengganggu geocoding
        address = ' '.join([x for x in address.split() if not ('RT' in x or 'RW' in x)])
        
        # Format alamat lengkap
        full_address = f"{address}, {kecamatan}, Tangerang Selatan, Banten, Indonesia"
        return full_address

    def get_coordinates(self, address: str, kecamatan: str) -> Optional[Tuple[float, float]]:
        """
        Mencoba mendapatkan koordinat menggunakan multiple providers
        """
        clean_addr = self.clean_address(address, kecamatan)
        
        for provider in self.providers:
            try:
                g = geocoder.get(clean_addr, provider=provider)
                if g.ok:
                    time.sleep(self.delay)
                    return (g.lat, g.lng)
            except Exception as e:
                print(f"Error with {provider}: {str(e)}")
                continue
        
        return None

def analyze_and_update_coordinates():
    """
    Menganalisis dan mengupdate koordinat menggunakan multiple geocoding services
    """
    try:
        # Baca CSV
        df = pd.read_csv('merchants_tangsel.csv', 
                        sep=';',
                        encoding='utf-8',
                        on_bad_lines='warn')
        
        print("\n=== ANALISIS DATA AWAL ===")
        print(f"Total data: {len(df)}")
        
        # Bersihkan data koordinat
        df['Koordinat'] = df['Koordinat'].astype(str).str.replace('"', '').str.strip()
        
        # Pisahkan koordinat
        df[['Latitude', 'Longitude']] = df['Koordinat'].str.split(',', expand=True)
        
        # Bersihkan nilai koordinat - menggunakan raw string untuk menghindari warning
        df['Latitude'] = df['Latitude'].str.replace(r'[^\d.-]', '', regex=True)
        df['Longitude'] = df['Longitude'].str.replace(r'[^\d.-]', '', regex=True)
        
        # Konversi ke float
        df['Latitude'] = pd.to_numeric(df['Latitude'], errors='coerce')
        df['Longitude'] = pd.to_numeric(df['Longitude'], errors='coerce')
        
        # Identifikasi data NULL
        null_coords = df[df['Latitude'].isna() | df['Longitude'].isna()].copy()
        print(f"\nData dengan koordinat NULL: {len(null_coords)}")
        
        if len(null_coords) > 0:
            print("\nMemulai proses update koordinat...")
            
            # Inisialisasi service
            geocoder_service = MultiGeocodingService()
            updated_count = 0
            
            # Proses setiap baris NULL
            for idx in null_coords.index:
                alamat = df.loc[idx, 'Alamat']
                kecamatan = df.loc[idx, 'Kecamatan']
                
                print(f"\nMencari koordinat untuk: {alamat}")
                
                # Coba dapatkan koordinat
                result = geocoder_service.get_coordinates(alamat, kecamatan)
                
                if result:
                    df.loc[idx, 'Latitude'] = result[0]
                    df.loc[idx, 'Longitude'] = result[1]
                    updated_count += 1
                    print(f"Berhasil: {result[0]}, {result[1]}")
                else:
                    print("Tidak ditemukan")
            
            print(f"\nBerhasil mengupdate {updated_count} dari {len(null_coords)} data koordinat NULL")
            
            # Update kolom Koordinat
            df['Koordinat'] = df.apply(
                lambda x: f"{x['Latitude']}, {x['Longitude']}" 
                if pd.notnull(x['Latitude']) and pd.notnull(x['Longitude']) 
                else x['Koordinat'],
                axis=1
            )
            
            # Simpan hasil
            df.to_csv('merchants_tangsel_updated.csv', sep=';', index=False)
            print("\nData telah disimpan ke merchants_tangsel_updated.csv")
            
            # Simpan data yang masih NULL
            still_null = df[df['Latitude'].isna() | df['Longitude'].isna()]
            if len(still_null) > 0:
                still_null.to_csv('koordinat_masih_null.csv', sep=';', index=False)
                print(f"\nMasih terdapat {len(still_null)} data dengan koordinat NULL")
                print("Data tersebut telah disimpan ke koordinat_masih_null.csv")
            
    except Exception as e:
        print(f"Error dalam pemrosesan data: {str(e)}")

if __name__ == "__main__":
    analyze_and_update_coordinates()


=== ANALISIS DATA AWAL ===
Total data: 1088

Data dengan koordinat NULL: 293

Memulai proses update koordinat...

Mencari koordinat untuk: JL. DUTA LESTARI


Status code 403 from https://nominatim.openstreetmap.org/search: ERROR - 403 Client Error: Forbidden for url: https://nominatim.openstreetmap.org/search?q=Jalan+DUTA+LESTARI%2C+CIPUTAT%2C+Tangerang+Selatan%2C+Banten%2C+Indonesia&format=jsonv2&addressdetails=1&limit=1


Error with bing: Provide API Key
Berhasil: -6.317860776061319, 106.75162488233585

Mencari koordinat untuk: JL. CIMANDIRI VI


Status code 403 from https://nominatim.openstreetmap.org/search: ERROR - 403 Client Error: Forbidden for url: https://nominatim.openstreetmap.org/search?q=Jalan+CIMANDIRI+VI%2C+CIPUTAT%2C+Tangerang+Selatan%2C+Banten%2C+Indonesia&format=jsonv2&addressdetails=1&limit=1


Error with bing: Provide API Key
Berhasil: -6.331229121855491, 106.75559835058559

Mencari koordinat untuk: JL CIMANDIRI RAYA RT 001 / RW 008


...

Error with bing: Provide API Key
Berhasil: -6.352845917620158, 106.68545403829097

Mencari koordinat untuk: Komp Puri Serpong Blok D4 no.7


Status code 403 from https://nominatim.openstreetmap.org/search: ERROR - 403 Client Error: Forbidden for url: https://nominatim.openstreetmap.org/search?q=Komp+Puri+Serpong+Blok+D4+no.7%2C+SETU%2C+Tangerang+Selatan%2C+Banten%2C+Indonesia&format=jsonv2&addressdetails=1&limit=1


Error with bing: Provide API Key
Berhasil: -6.349685898213508, 106.6819560186388

Mencari koordinat untuk: Kp setu Rt 015/004 keurahan setu


Status code 403 from https://nominatim.openstreetmap.org/search: ERROR - 403 Client Error: Forbidden for url: https://nominatim.openstreetmap.org/search?q=Kp+setu+Rt+015%2F004+keurahan+setu%2C+SETU%2C+Tangerang+Selatan%2C+Banten%2C+Indonesia&format=jsonv2&addressdetails=1&limit=1


Error with bing: Provide API Key
Berhasil: -6.343362046274239, 106.67840989147153

Berhasil mengupdate 293 dari 293 data koordinat NULL

Data telah disimpan ke merchants_tangsel_updated.csv


In [28]:
import pandas as pd
import numpy as np

def analyze_duplicate_coordinates():
    """
    Menganalisis koordinat duplikat dari dataset yang telah diupdate
    """
    try:
        # Baca file hasil update
        df = pd.read_csv('merchants_tangsel_updated.csv', 
                        sep=';',
                        encoding='utf-8',
                        on_bad_lines='warn')
        
        print("\n=== ANALISIS KOORDINAT DUPLIKAT ===")
        print(f"Total data: {len(df)}")
        
        # Bersihkan data koordinat
        df['Koordinat'] = df['Koordinat'].astype(str).str.replace('"', '').str.strip()
        
        # Pisahkan koordinat menjadi latitude dan longitude
        df[['Latitude', 'Longitude']] = df['Koordinat'].str.split(',', expand=True)
        
        # Bersihkan nilai latitude dan longitude
        df['Latitude'] = df['Latitude'].str.replace(r'[^\d.-]', '', regex=True)
        df['Longitude'] = df['Longitude'].str.replace(r'[^\d.-]', '', regex=True)
        
        # Konversi ke float
        df['Latitude'] = pd.to_numeric(df['Latitude'], errors='coerce')
        df['Longitude'] = pd.to_numeric(df['Longitude'], errors='coerce')
        
        # Identifikasi data dengan koordinat valid
        valid_coords = df[df['Latitude'].notna() & df['Longitude'].notna()].copy()
        print(f"\nData dengan koordinat valid: {len(valid_coords)}")
        
        # Round koordinat ke 6 desimal untuk menghindari perbedaan kecil
        valid_coords['Latitude'] = valid_coords['Latitude'].round(6)
        valid_coords['Longitude'] = valid_coords['Longitude'].round(6)
        
        # Cari koordinat duplikat
        duplicate_mask = valid_coords.duplicated(subset=['Latitude', 'Longitude'], keep=False)
        duplicates = valid_coords[duplicate_mask].sort_values(['Latitude', 'Longitude'])
        
        print(f"\nDitemukan {len(duplicates)} data dengan koordinat duplikat")
        
        if len(duplicates) > 0:
            # Tambahkan kolom untuk menunjukkan grup duplikat
            duplicates['Duplicate_Group'] = duplicates.groupby(['Latitude', 'Longitude']).ngroup() + 1
            
            # Tampilkan contoh duplikat
            print("\nContoh data duplikat:")
            print(duplicates[['Kecamatan', 'Kelurahan', 'Nama Pemilik', 'Alamat', 'Latitude', 'Longitude', 'Duplicate_Group']].head())
            
            # Hitung statistik duplikat
            duplicate_stats = duplicates.groupby(['Latitude', 'Longitude']).size().value_counts()
            print("\nStatistik duplikat:")
            print("Jumlah koordinat yang muncul beberapa kali:")
            print(duplicate_stats)
            
            # Simpan data duplikat ke file
            duplicates.to_csv('koordinat_duplikat.csv', sep=';', index=False)
            print("\nData koordinat duplikat telah disimpan ke koordinat_duplikat.csv")
            
            # Analisis jarak antar merchant dengan koordinat sama
            print("\nAnalisis merchant dengan koordinat sama:")
            for group in duplicates['Duplicate_Group'].unique():
                group_data = duplicates[duplicates['Duplicate_Group'] == group]
                print(f"\nGrup {group} - {len(group_data)} merchant:")
                for idx, row in group_data.iterrows():
                    print(f"- {row['Nama Pemilik']} di {row['Alamat']}")
        
        # Tampilkan statistik tambahan
        print("\nStatistik Koordinat:")
        print("\nLatitude:")
        print(valid_coords['Latitude'].describe())
        print("\nLongitude:")
        print(valid_coords['Longitude'].describe())
        
        # Cek range koordinat yang valid untuk wilayah Tangerang Selatan
        # Approximate bounding box for Tangerang Selatan
        tangsel_bounds = {
            'lat_min': -6.35,
            'lat_max': -6.25,
            'lon_min': 106.65,
            'lon_max': 106.85
        }
        
        out_of_bounds = valid_coords[
            (valid_coords['Latitude'] < tangsel_bounds['lat_min']) |
            (valid_coords['Latitude'] > tangsel_bounds['lat_max']) |
            (valid_coords['Longitude'] < tangsel_bounds['lon_min']) |
            (valid_coords['Longitude'] > tangsel_bounds['lon_max'])
        ]
        
        if len(out_of_bounds) > 0:
            print(f"\nDitemukan {len(out_of_bounds)} data dengan koordinat di luar wilayah Tangerang Selatan")
            out_of_bounds.to_csv('koordinat_out_of_bounds.csv', sep=';', index=False)
            print("Data tersebut telah disimpan ke koordinat_out_of_bounds.csv")
        
    except Exception as e:
        print(f"Error dalam analisis data: {str(e)}")

if __name__ == "__main__":
    analyze_duplicate_coordinates()


=== ANALISIS KOORDINAT DUPLIKAT ===
Total data: 1088

Data dengan koordinat valid: 1088

Ditemukan 197 data dengan koordinat duplikat

Contoh data duplikat:
    Kecamatan    Kelurahan              Nama Pemilik  \
170   CIPUTAT  SERUA INDAH      ELSA SEPHIANA ZAFIRA   
171   CIPUTAT  SERUA INDAH           VICKRI QURAHMAN   
752   SERPONG       BUARAN                    SOPYAN   
773   SERPONG       CIATER  ZUBAIDAH BUDIPUSPITORINI   
850   SERPONG    RAWABUNTU                     HERRY   

                                       Alamat  Latitude   Longitude  \
170                           Jl suka Mulya 1 -6.378458  106.703565   
171                           Jl suka Mulya 1 -6.378458  106.703565   
752                          Jl.raya puspitek -6.378458  106.703565   
773                             Kp.rawa macek -6.378458  106.703565   
850  Ruko versailles blok fb no 18 sektor 1-6 -6.378458  106.703565   

     Duplicate_Group  
170                1  
171                1  
752      

In [1]:
import pandas as pd
import geocoder
import time
from typing import Optional, Tuple

class MultiGeocodingService:
    def __init__(self):
        self.providers = ['osm', 'bing', 'arcgis']
        self.delay = 1
    
    def clean_address(self, address: str, kecamatan: str) -> str:
        """
        Membersihkan dan memformat alamat
        """
        # Standardisasi format jalan
        address = address.replace('JL.', 'Jalan')
        address = address.replace('JL ', 'Jalan ')
        address = address.replace('Jl.', 'Jalan')
        address = address.replace('Jl ', 'Jalan ')
        
        # Hapus detail RT/RW
        address = ' '.join([x for x in address.split() if not ('RT' in x or 'RW' in x)])
        
        # Format alamat lengkap
        full_address = f"{address}, {kecamatan}, Tangerang Selatan, Banten, Indonesia"
        return full_address

    def get_coordinates(self, address: str, kecamatan: str) -> Optional[Tuple[float, float]]:
        """
        Mendapatkan koordinat dari alamat
        """
        clean_addr = self.clean_address(address, kecamatan)
        
        for provider in self.providers:
            try:
                g = geocoder.get(clean_addr, provider=provider)
                if g.ok:
                    time.sleep(self.delay)
                    return (g.lat, g.lng)
            except Exception as e:
                print(f"Error with {provider}: {str(e)}")
                continue
        
        return None

def update_duplicate_coordinates():
    """
    Mengupdate koordinat yang duplikat dengan alamat berbeda
    """
    try:
        # Baca file hasil update sebelumnya
        df = pd.read_csv('merchants_tangsel_updated.csv', 
                        sep=';',
                        encoding='utf-8',
                        on_bad_lines='warn')
        
        print("\n=== ANALISIS DAN UPDATE KOORDINAT DUPLIKAT ===")
        print(f"Total data: {len(df)}")
        
        # Bersihkan data koordinat
        df['Koordinat'] = df['Koordinat'].astype(str).str.replace('"', '').str.strip()
        
        # Pisahkan koordinat
        df[['Latitude', 'Longitude']] = df['Koordinat'].str.split(',', expand=True)
        
        # Bersihkan nilai koordinat
        df['Latitude'] = df['Latitude'].str.replace(r'[^\d.-]', '', regex=True)
        df['Longitude'] = df['Longitude'].str.replace(r'[^\d.-]', '', regex=True)
        
        # Konversi ke float
        df['Latitude'] = pd.to_numeric(df['Latitude'], errors='coerce')
        df['Longitude'] = pd.to_numeric(df['Longitude'], errors='coerce')
        
        # Round koordinat untuk perbandingan
        df['Latitude_rounded'] = df['Latitude'].round(6)
        df['Longitude_rounded'] = df['Longitude'].round(6)
        
        # Identifikasi koordinat duplikat
        duplicates = df[df.duplicated(subset=['Latitude_rounded', 'Longitude_rounded'], keep=False)].copy()
        
        print(f"\nDitemukan {len(duplicates)} data dengan koordinat duplikat")
        
        if len(duplicates) > 0:
            # Inisialisasi geocoding service
            geocoder_service = MultiGeocodingService()
            updated_count = 0
            
            # Grup berdasarkan koordinat
            duplicate_groups = duplicates.groupby(['Latitude_rounded', 'Longitude_rounded'])
            
            print("\nMemulai proses update koordinat duplikat...")
            
            for (lat, lon), group in duplicate_groups:
                # Jika ada alamat berbeda dalam grup yang sama
                unique_addresses = group['Alamat'].unique()
                
                if len(unique_addresses) > 1:
                    print(f"\nMengupdate koordinat untuk grup {lat}, {lon}")
                    print(f"Jumlah alamat berbeda: {len(unique_addresses)}")
                    
                    # Proses setiap alamat berbeda kecuali yang pertama
                    first_address = unique_addresses[0]
                    for address in unique_addresses[1:]:
                        # Dapatkan index untuk alamat ini
                        address_indices = group[group['Alamat'] == address].index
                        
                        print(f"\nAlamat berbeda ditemukan:")
                        print(f"Original: {first_address}")
                        print(f"Perlu diupdate: {address}")
                        
                        # Cari koordinat baru
                        for idx in address_indices:
                            kecamatan = df.loc[idx, 'Kecamatan']
                            result = geocoder_service.get_coordinates(address, kecamatan)
                            
                            if result:
                                df.loc[idx, 'Latitude'] = result[0]
                                df.loc[idx, 'Longitude'] = result[1]
                                df.loc[idx, 'Koordinat'] = f"{result[0]}, {result[1]}"
                                updated_count += 1
                                print(f"Berhasil update: {result[0]}, {result[1]}")
                            else:
                                print(f"Gagal mendapatkan koordinat baru untuk: {address}")
            
            print(f"\nBerhasil mengupdate {updated_count} data koordinat duplikat")
            
            # Simpan hasil
            output_file = 'merchants_tangsel_final.csv'
            df.drop(['Latitude_rounded', 'Longitude_rounded'], axis=1).to_csv(output_file, sep=';', index=False)
            print(f"\nData telah disimpan ke {output_file}")
            
            # Cek ulang duplikat setelah update
            df['Latitude'] = pd.to_numeric(df['Latitude'], errors='coerce')
            df['Longitude'] = pd.to_numeric(df['Longitude'], errors='coerce')
            final_duplicates = df[df.duplicated(subset=['Latitude', 'Longitude'], keep=False)]
            
            print("\nStatistik akhir:")
            print(f"Total data: {len(df)}")
            print(f"Data dengan koordinat duplikat tersisa: {len(final_duplicates)}")
            
            # Simpan data duplikat tersisa (yang memiliki alamat sama)
            if len(final_duplicates) > 0:
                final_duplicates.to_csv('koordinat_duplikat_tersisa.csv', sep=';', index=False)
                print("\nData koordinat duplikat tersisa telah disimpan ke koordinat_duplikat_tersisa.csv")
        
    except Exception as e:
        print(f"Error dalam pemrosesan data: {str(e)}")

if __name__ == "__main__":
    update_duplicate_coordinates()


=== ANALISIS DAN UPDATE KOORDINAT DUPLIKAT ===
Total data: 1088

Ditemukan 197 data dengan koordinat duplikat

Memulai proses update koordinat duplikat...

Mengupdate koordinat untuk grup -6.378458, 106.703565
Jumlah alamat berbeda: 10

Alamat berbeda ditemukan:
Original: Jl suka Mulya 1
Perlu diupdate: Jl.raya puspitek


Status code 403 from https://nominatim.openstreetmap.org/search: ERROR - 403 Client Error: Forbidden for url: https://nominatim.openstreetmap.org/search?q=Jalanraya+puspitek%2C+SERPONG%2C+Tangerang+Selatan%2C+Banten%2C+Indonesia&format=jsonv2&addressdetails=1&limit=1


Error with bing: Provide API Key
Berhasil update: -6.313423460465434, 106.660501996923

Alamat berbeda ditemukan:
Original: Jl suka Mulya 1
Perlu diupdate: Kp.rawa macek


Status code 403 from https://nominatim.openstreetmap.org/search: ERROR - 403 Client Error: Forbidden for url: https://nominatim.openstreetmap.org/search?q=Kp.rawa+macek%2C+SERPONG%2C+Tangerang+Selatan%2C+Banten%2C+Indonesia&format=jsonv2&addressdetails=1&limit=1


Error with bing: Provide API Key
Berhasil update: -6.3189062, 106.6683414

Alamat berbeda ditemukan:
Original: Jl suka Mulya 1
Perlu diupdate: Ruko versailles blok fb no 18 sektor 1-6


...

Error with bing: Provide API Key
Berhasil update: -6.35473129968361, 106.75544126309335

Alamat berbeda ditemukan:
Original: Pondok Cipayung Mas Jl. Gelatik Blok E No.5 Tangerang Selatan
Perlu diupdate: Jl. Kayu Manis Pd. Cabe Raya


Status code 403 from https://nominatim.openstreetmap.org/search: ERROR - 403 Client Error: Forbidden for url: https://nominatim.openstreetmap.org/search?q=Jalan+Kayu+Manis+Pd.+Cabe+Raya%2C+PAMULANG%2C+Tangerang+Selatan%2C+Banten%2C+Indonesia&format=jsonv2&addressdetails=1&limit=1


Error with bing: Provide API Key
Berhasil update: -6.318097364264928, 106.73142790739055

Alamat berbeda ditemukan:
Original: Pondok Cipayung Mas Jl. Gelatik Blok E No.5 Tangerang Selatan
Perlu diupdate: Jl Raya Bintaro Sektor IX


Status code 403 from https://nominatim.openstreetmap.org/search: ERROR - 403 Client Error: Forbidden for url: https://nominatim.openstreetmap.org/search?q=Jalan+Raya+Bintaro+Sektor+IX%2C+PONDOK+AREN%2C+Tangerang+Selatan%2C+Banten%2C+Indonesia&format=jsonv2&addressdetails=1&limit=1


Error with bing: Provide API Key
Berhasil update: -6.267317108825859, 106.75254799496584

Berhasil mengupdate 141 data koordinat duplikat

Data telah disimpan ke merchants_tangsel_final.csv

Statistik akhir:
Total data: 1088
Data dengan koordinat duplikat tersisa: 149

Data koordinat duplikat tersisa telah disimpan ke koordinat_duplikat_tersisa.csv
